In [0]:
import tensorflow
import csv
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam, SGD
from keras import optimizers
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
import os
from skimage import transform, io
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

import os
%matplotlib inline

np.random.seed(15)

#Data 불러오기 & Data Normalization

In [0]:
data = '/content/drive/My Drive/Faces Project/fer2013.csv'

In [0]:
x_train = []    #모든 train image set
y_train = []    #모든 train label set

x_test = []     #test image set
y_test=[]       #test label set

a = np.zeros(7) #클래스별로 이미지가 몇개씩 있는지 확인할 변수

In [0]:
# train image를 클래스 별로 분류
angry =[]
angry_label=[]
disgust = []
disgust_label = []
fear =[]
fear_label =[]
happy =[]
happy_label =[]
sad =[]
sad_label =[]
surprise =[]
surprise_label =[]
neutral =[]
neutral_label =[]

In [0]:
#파일을 불러와서 train/test 셋으로 나눈다
#train 셋을 클래스 별로 분류한다
with open(data) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row_id, row in enumerate(csv_reader):
        # skip header
        if row_id == 0:
            continue
        
        label = np.zeros(7) + 0.01
        label[int(row[0])] = 0.99
        image = list(map(int, row[1].split(' ')))
        
        if row[2] == 'Training':
            if int(row[0]) == 0:
                angry.append(image)
                angry_label.append(label)
            elif int(row[0]) == 1:
                disgust.append(image)
                disgust_label.append(label)
            elif int(row[0]) == 2:
                fear.append(image)
                fear_label.append(label)
            elif int(row[0]) == 3:
                happy.append(image)
                happy_label.append(label)
            elif int(row[0]) == 4:
                sad.append(image)
                sad_label.append(label)
            elif int(row[0]) == 5:
                surprise.append(image)
                surprise_label.append(label)
            elif int(row[0]) == 6:
                neutral.append(image)
                neutral_label.append(label)
            a[int(row[0])] =  a[int(row[0])]+1
        elif row[2] == 'Testing': 
            y_test.append(label)
            x_test.append(image)

In [0]:
angry= np.asarray(angry, dtype = np.float32).reshape(-1, 48, 48, 1)
angry_label = np.asarray(angry_label, dtype = np.float32)
disgust= np.asarray(disgust, dtype = np.float32).reshape(-1, 48, 48, 1)
disgust_label = np.asarray(disgust_label, dtype = np.float32)
fear= np.asarray(fear, dtype = np.float32).reshape(-1, 48, 48, 1)
fear_label = np.asarray(fear_label, dtype = np.float32)
happy= np.asarray(happy, dtype = np.float32).reshape(-1, 48, 48, 1)
happy_label = np.asarray(happy_label, dtype = np.float32)
sad= np.asarray(sad, dtype = np.float32).reshape(-1, 48, 48, 1)
sad_label = np.asarray(sad_label, dtype = np.float32)
surprise= np.asarray(surprise, dtype = np.float32).reshape(-1, 48, 48, 1)
surprise_label = np.asarray(surprise_label, dtype = np.float32)
neutral= np.asarray(neutral, dtype = np.float32).reshape(-1, 48, 48, 1)
neutral_label = np.asarray(neutral_label, dtype = np.float32)

x_test = np.asarray(x_test, dtype = np.float32).reshape(-1, 48, 48, 1)
y_test = np.asarray(y_test, dtype = np.float32)

In [0]:

x_test = np.asarray(x_test, dtype = np.float32).reshape(-1, 48, 48, 1)
y_test = np.asarray(y_test, dtype = np.float32)

In [0]:
#normalization
angry /= 255.0
disgust /= 255.0
fear /= 255.0
happy /= 255.0
sad /= 255.0
surprise /= 255.0
neutral /= 255.0
x_test /= 255.0

In [0]:
#클래스 별로 분류한 데이터를 하나의 train set으로 합쳐 모델을 학습시킴
x_train = np.concatenate((angry, disgust, fear, happy, sad, surprise, neutral))
y_train = np.concatenate((angry_label, disgust_label, fear_label,happy_label, sad_label, surprise_label, neutral_label))

In [0]:
#train set을 validation을 위한 데이터와 train을 위한 데이터로 나눠준다
#random_state 하면 실행 시킬 때 마다 같은 기준으로 나눠줌!
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1,random_state = 64)

In [0]:
print("x_train.shape: ", x_train.shape)
print("y_train.shape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
print("x_val.shape: ", x_val.shape)
print("y_val.shape:", y_val.shape)

#이미지 출력해보기

In [0]:
emotions = {
    0: 'Angry', 
    1: 'Disgust', 
    2: 'Fear', 
    3: 'Happy',
    4: 'Sad', 
    5: 'Surprise', 
    6: 'Neutral'
}

In [0]:
#원본이미지 갯수
total=0;
for i in range(7):
    print(emotions[i],':',a[i])
    total+=a[i]
print('-'*20)
print('total: ', total)

In [0]:
index = 1000
print('Label array:', y_train[index], '\nLabel:', np.argmax(y_train[index]),
      '\nEmotion:', emotions[np.argmax(y_train[index])],
      '\nImage shape:', x_train[index].shape)
plt.imshow(x_train[index].reshape(48, 48), cmap='gray')
plt.show()
image = x_train[2].reshape(48, 48)

# First Model
- 처음 주어진 european_dataset만 가지고 훈련시킨 모델

In [0]:
#모델 생성
def make_model():
    #흑백이미지로 인풋을 넣음
    input_shape = (48, 48, 1)
   
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), input_shape=input_shape, activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Dropout(0.55))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='softmax'))
    return model

In [0]:
#모델 컴파일 함수 (파라미터로 모델과 ,러닝rate가 들어감)
def model_compile(model, rate):

    #Compliling the model with adam optimixer and categorical crossentropy loss
    model.compile(loss='categorical_crossentropy',
                optimizer=Adam(lr=rate, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
                metrics=['accuracy'])


In [0]:
#valid 데이터와 train데이터의 accuracy와 loss의 양상을 그래프로 시각화
def show_history(train_history):
    acc = train_history.history['accuracy']
    val_acc = train_history.history['val_accuracy']
    loss = train_history.history['loss']
    val_loss = train_history.history['val_loss']

    epochs = range(len(acc))

  

    plt.plot(epochs, loss, 'bo', label='Training loss', c = 'blue') 
    plt.plot(epochs, val_loss, 'b', label='Validation loss' ,c='red')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show() 
    plt.plot(epochs, acc, 'bo', label='Training acc', c = 'blue')
    plt.plot(epochs, val_acc, 'b', label='Validation acc', c= 'red')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

In [0]:
# tset data로 모델 평가
def model_eval(model):
    loss, accuracy = model.evaluate(x_test, y_test, verbose = 1)
    print("loss: ",loss)
    print("accuracy: ", accuracy*100,'%')

In [0]:
#fer2013으로 주어진 유러피안 데이터만 가지고 학습시킨모델
model = make_model()
model_compile(model, 0.001)
history = model.fit(x_train, y_train,
        batch_size=128,
        epochs=30,
        validation_data=(x_val, y_val),
        verbose=1)
show_history(history)
model_eval(model)

In [0]:
show_history(history)
model_eval(model)

In [0]:
model_eval(model)
#61.32

In [0]:
model.save('/content/drive/My Drive/Faces Project/model1.h5')

# Data Augumentation 데이터 증강
- 더 많은 이미지를 학습하여 overfitting방지
- ImageDataGenerator를 이용


## 1 - 이미지 반전시키기

In [0]:
#이미지를 좌우로 반전시키는 함수
def data_transpose(x_train, y_train):
    x = np.zeros_like(x_train)
    y = np.zeros_like(y_train)
    for i in range(x_train.shape[0]):
       for j in range(x_train.shape[1]):
           for k in range(x_train.shape[2]): 
               x[i,j,k,0] = x_train[i,j,47-k,0]
    y = y_train
    return x , y

In [0]:
#클래스별로 이미지 좌우 반전
angry_t , angry_label_t = data_transpose(angry, angry_label)
disgust_t , disgust_label_t = data_transpose(disgust, disgust_label)
fear_t , fear_label_t = data_transpose(fear, fear_label)
happy_t , happy_label_t = data_transpose(happy, happy_label)
sad_t ,sad_label_t = data_transpose(sad, sad_label)
surprise_t , suprise_label_t = data_transpose(surprise, surprise_label)
neutral_t ,neutral_label_t = data_transpose(neutral, neutral_label)

In [0]:
# 원본 이미지
index = 1000
print('Label array:', angry_label[index], '\nLabel:',(angry_label[index]),
      '\nEmotion:', emotions[np.argmax(y_train[index])],
      '\nImage shape:', angry[index].shape)
plt.imshow(angry[index].reshape(48, 48), cmap='gray')
plt.show()
image = angry[2].reshape(48, 48)

In [0]:
# 좌우 반전된 이미지
index = 1000
print('Label array:', angry_label_t[index], '\nLabel:',(angry_label_t[index]),
      '\nEmotion:', emotions[np.argmax(y_train[index])],
      '\nImage shape:', angry_t[index].shape)
plt.imshow(angry_t[index].reshape(48, 48), cmap='gray')
plt.show()
image = angry_t[2].reshape(48, 48)

In [0]:
#원본 이미지와 좌우 반전된 이미지를 합치기
angry = np.concatenate((angry,angry_t))
disgust = np.concatenate((disgust,disgust_t))
fear = np.concatenate((fear, fear_t))
happy = np.concatenate((happy, happy_t))
sad = np.concatenate((sad, sad_t))
surprise = np.concatenate((surprise, surprise_t))
neutral = np.concatenate((neutral, neutral_t))


In [0]:
angry_label = np.concatenate((angry_label_t, angry_label))
disgust_label = np.concatenate((disgust_label_t, disgust_label))
fear_label = np.concatenate((fear_label_t, fear_label))
happy_label = np.concatenate((happy_label_t, happy_label))
sad_label = np.concatenate((sad_label_t, sad_label))
surprise_label = np.concatenate((suprise_label_t, surprise_label))
neutral_label = np.concatenate((neutral_label_t, neutral_label))


## 2 - ImageDataGenerator이용하여 회전, 확대, 밝기 조절, 좌우 반전


- 현재 데이터가 클래스 별로 불균형하게 있음

In [0]:
from keras.preprocessing.image import ImageDataGenerator
np.random.seed(15)

datagen = ImageDataGenerator(
    featurewise_center=False,  
    samplewise_center=False,  
    featurewise_std_normalization=False,  
    samplewise_std_normalization=False,  
    zca_whitening=False,  

    rotation_range = 8, #8도 회전 범위
    width_shift_range = 0.001,  #좌우로 이동범위
    height_shift_range = 0.001, #위아래로 이동범위
    zoom_range = 0.2,       #확대축소범위
    shear_range = 0.02,     #망가짐?범위
    horizontal_flip = True, #좌우 반전
    vertical_flip = False,  
    fill_mode = 'nearest', #가까운 픽셀로 값을 채움

)

- 모든 클래스마다 20000만개의 데이터로 만들기

In [0]:
#이미지를 생성하는 함수
def generate_image(images, label):
    copy_label=[]
    arr=[]
    num = 20000-images.shape[0] #2만개를 만들어야해서 2만에서 클래스별 이미지를 뺀 만큼만 이미지 생성
    it = datagen.flow(images, batch_size = 1)   
    fig = plt.figure(figsize = (48,48))
    for i in range(num):
        #plt.subplot (10,3,i+1)
        batch = it.next()
        arr.append(batch.reshape(48,48,1))  #생성된 이미지를 arr에 저장
        copy_label.append(label)
        #plt.imshow(batch.reshape(48,48))
    #plt.show()
    arr = np.array(arr)
    copy_label = np.array(copy_label)
    return arr , copy_label

In [0]:
#클래스별로 이미지 증대
generate_angry, label0 = generate_image(angry, angry_label[0])

generate_disgust, label1= generate_image(disgust, disgust_label[0])

generate_fear, label2 = generate_image(fear, fear_label[0])

generate_happy, label3 = generate_image(happy, happy_label[0])

generate_sad, label4 = generate_image(sad, sad_label[0])

generate_surprise, label5 = generate_image(surprise, surprise_label[0])

generate_neutral, label6 =generate_image(neutral, neutral_label[0])

In [0]:
# 클래스별로 이미지를 합치기
angry = np.concatenate((angry, generate_angry))
disgust = np.concatenate((disgust, generate_disgust))
fear = np.concatenate((fear, generate_fear))
happy = np.concatenate((happy, generate_happy))
sad = np.concatenate((sad, generate_sad))
surprise = np.concatenate((surprise, generate_surprise))
neutral = np.concatenate((neutral, generate_neutral))


In [0]:
angry_label = np.concatenate((angry_label, label0))
disgust_label = np.concatenate((disgust_label, label1))
fear_label = np.concatenate((fear_label, label2))
happy_label = np.concatenate((happy_label, label3))
sad_label = np.concatenate((sad_label, label4))
surprise_label = np.concatenate((surprise_label, label5))
neutral_label = np.concatenate((neutral_label, label6))


In [0]:
#모든 클래스를 합치기
x_train =np.concatenate((angry, disgust, fear, happy, sad, surprise, neutral))
y_train = np.concatenate((angry_label, disgust_label, fear_label, happy_label, sad_label, surprise_label, neutral_label))

In [0]:
print("angry.shape: "  ,angry.shape)
print("disgust.shape: ", disgust.shape )
print("fear.shape: ",fear.shape)
print("happy.shape: ", happy.shape)
print("sad.shape: ", sad.shape)
print("surprise.shape: ",surprise.shape)
print("neutral.shape: ", neutral.shape)

In [0]:
print("x_train.shape: ", x_train.shape)
print("y_train.shape: ", y_train.shape)

# Second Model
- 14만개의 데이터 셋으로 훈련

In [0]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience =25)    #val loss가 25동안 감소안하면 학습 종료

In [0]:

model2 = make_model()
model_compile(model2,0.0001)    #lr = 0.0001
history = model2.fit(x_train, y_train,
        batch_size=256,
        epochs=200,
        validation_data=(x_val, y_val),
        callbacks = [early_stopping],   #조기종료함수
        verbose=1)
show_history(history)
model_eval(model2)

In [0]:
#모델 저장
model2.save('/content/drive/My Drive/Faces Project/model2.h5')

# 한국인 데이터 셋으로 모델 평가해보기

In [0]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import scipy as sp
import glob
import cv2


#한국인 이미지를 48*48사이즈 , 흑백이미지로 불러와서 저장
imgs = glob.glob('/content/drive/My Drive/Faces Project/Images_12161235/*.png')
koreans = []
for path in imgs:
    img = cv2.imread(path)
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img3 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    img4 = cv2.resize(img3, (48,48), interpolation = cv2.INTER_LINEAR)
    img5 = img_to_array(img4)
    koreans.append(img5)

In [0]:
koreans = np.asarray(koreans, dtype="float32")

In [0]:
koreans /= 255.0

In [0]:
index = 5
plt.imshow(koreans[index].reshape(48,48), cmap='gray')
plt.show()

In [0]:
#한국인 라벨링한 파일 읽어오기
data = '/content/drive/My Drive/Faces Project/korean_face_dataset_12161235.csv'
koreans_label=[]
with open(data) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row_id, row in enumerate(csv_reader):
        # skip header
        if row_id == 0:
            continue
        
        label = np.zeros(7) + 0.01
        label[int(row[0])] = 0.99
        koreans_label.append(label)

In [0]:
koreans_label = np.asarray(koreans_label, dtype= "float32")

In [0]:
#한국인 데이터를 train과 test로 나누기
kx_train, kx_test, ky_train, ky_test = train_test_split(koreans, koreans_label, test_size=0.1,random_state = 64)

In [0]:
#학습된 모델 불러오기
model2 = load_model('/content/drive/My Drive/Faces Project/model2.h5')

In [0]:
#한국인 데이터로 저장된 모델 평가
loss, accuracy = model2.evaluate(kx_test, ky_test, verbose = 1)
print("loss: ",loss)
print("accuracy: ",accuracy*100,'%')

# 한국인 데이터로 모델 훈련시키기

In [0]:
model22 = make_model()
model_compile(model22,0.0001)    

history = model22.fit(kx_train, ky_train,
        batch_size=256,
        epochs=200,
        verbose=1)
loss, accuracy = model22.evaluate(kx_test, ky_test, verbose = 1)
print("loss: ",loss)
print("accuracy: ",accuracy*100,'%')

In [0]:
model_eval(model22)

#공유폴더에 있는 한국인 데이터들 불러오기

In [0]:
imgs = glob.glob('/content/drive/My Drive/Faces Project/Images_12170151/*.png')
koreans2 = []
for path in imgs:
    img = cv2.imread(path)
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img3 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    img4 = cv2.resize(img3, (48,48), interpolation = cv2.INTER_LINEAR)
    img5 = img_to_array(img4)
    koreans2.append(img5)

koreans2 = np.asarray(koreans2, dtype="float32")
koreans2 /= 255.0

data = '/content/drive/My Drive/Faces Project/korean_face_dataset_12170151.csv'
koreans_label2=[]
with open(data) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row_id, row in enumerate(csv_reader):
        # skip header
        if row_id == 0:
            continue
        
        label = np.zeros(7) + 0.01
        label[int(row[0])] = 0.99
        koreans_label2.append(label)
koreans_label2 = np.asarray(koreans_label2, dtype= "float32")

In [0]:
imgs = glob.glob('/content/drive/My Drive/Faces Project/Images_12131569/*.png')
koreans3 = []
for path in imgs:
    img = cv2.imread(path)
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img3 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    img4 = cv2.resize(img3, (48,48), interpolation = cv2.INTER_LINEAR)
    img5 = img_to_array(img4)
    koreans3.append(img5)

koreans3 = np.asarray(koreans3, dtype="float32")
koreans3/= 255.0

data = '/content/drive/My Drive/Faces Project/korean_face_dataset_12131569.csv'
koreans_label3=[]
with open(data) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row_id, row in enumerate(csv_reader):
        # skip header
        if row_id == 0:
            continue
        
        label = np.zeros(7) + 0.01
        label[int(row[0])] = 0.99
        koreans_label3.append(label)
koreans_label3 = np.asarray(koreans_label3, dtype= "float32")

In [0]:
imgs = glob.glob('/content/drive/My Drive/Faces Project/Images_12181617/*.png')
koreans4 = []
for path in imgs:
    img = cv2.imread(path)
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img3 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    img4 = cv2.resize(img3, (48,48), interpolation = cv2.INTER_LINEAR)
    img5 = img_to_array(img4)
    koreans4.append(img5)

koreans4 = np.asarray(koreans4, dtype="float32")
koreans4 /= 255.0

data = '/content/drive/My Drive/Faces Project/korean_face_dataset_12181617.csv'
koreans_label4=[]
with open(data) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row_id, row in enumerate(csv_reader):
        # skip header
        if row_id == 0:
            continue
        
        label = np.zeros(7) + 0.01
        label[int(row[0])] = 0.99
        koreans_label4.append(label)
koreans_label4 = np.asarray(koreans_label4, dtype= "float32")

In [0]:
imgs = glob.glob('/content/drive/My Drive/Faces Project/Images_12181617/*.png')
koreans5 = []
for path in imgs:
    img = cv2.imread(path)
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img3 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    img4 = cv2.resize(img3, (48,48), interpolation = cv2.INTER_LINEAR)
    img5 = img_to_array(img4)
    koreans5.append(img5)

koreans5 = np.asarray(koreans5, dtype="float32")
koreans5 /= 255.0

data = '/content/drive/My Drive/Faces Project/korean_face_dataset_12181617.csv'
koreans_label5=[]
with open(data) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row_id, row in enumerate(csv_reader):
        # skip header
        if row_id == 0:
            continue
        
        label = np.zeros(7) + 0.01
        label[int(row[0])] = 0.99
        koreans_label5.append(label)
koreans_label5 = np.asarray(koreans_label5, dtype= "float32")

In [0]:
#모든 한국인 데이터 합치기
kor = np.concatenate((koreans5, koreans4, koreans3, koreans2, kx_train))
kor_label = np.concatenate((koreans_label5, koreans_label4, koreans_label3, koreans_label2, ky_train))

In [0]:
del koreans5
del koreans4
del koreans3
del koreans2

In [0]:
#모든 한국인 데이터를 합쳐서 훈련시킨 모델
model22 = make_model()
model_compile(model22,0.0001)    

history = model22.fit(kor, kor_label,
        batch_size=256,
        epochs=200,
        verbose=1)
loss, accuracy = model22.evaluate(kx_test, ky_test, verbose = 1)
print("loss: ",loss)
print("accuracy: ",accuracy*100,'%')

In [0]:
model_eval(model22)

In [0]:
#모든 한국인 데이터를 합치고 Argumentaion해서 훈련시킨 모델
model222 = make_model()
model_compile(model222,0.0001)    
datagen.fit(kor)

history = model222.fit_generator(datagen.flow(kor, kor_label, batch_size =128,),
        steps_per_epochs = kor.shape[0]/128, epochs = 200, verbose=1)
        epochs=200,
        verbose=1)
loss, accuracy = model222.evaluate(kx_test, ky_test, verbose = 1)
print("loss: ",loss)
print("accuracy: ",accuracy*100,'%')

# 14만개의 유러피안데이터와 40배 부풀린 한국인 데이터로 모델 학습

In [0]:
#모델 불러오기
model2 = load_model('/content/drive/My Drive/Faces Project/model2.h5')

In [0]:
#이미지를 생성하는 함수
#이미지를 한개씩 받아서 40배로 늘려줌
def generate_image2(images, label):
    copy_label=[]
    arr=[]
    images = np.expand_dims(images, axis=0) #(48,48,1)차원에서 0번째에 차원하나 추가시킴
    it = datagen.flow(images, batch_size = 1)   
    #fig = plt.figure(figsize = (48,48))
    for i in range(50):     #40개씩 생성
        #plt.subplot (10,3,i+1)
        batch = it.next()
        arr.append(batch.reshape(48,48,1))  #생성된 이미지를 arr에 저장
        copy_label.append(label)
        #plt.imshow(batch.reshape(48,48))
    #plt.show()
    arr = np.array(arr)
    copy_label = np.array(copy_label)
    return arr , copy_label

In [0]:
new_k =[]
new_k_lable=[]
new_k.append(kor[0])
new_k_lable.append(kor_label[0])
new_k = np.array(new_k)
new_k_lable = np.array(new_k_lable)


In [0]:
# 공유폴더의 한국인 데이터 argument하기 
for i in range(kor.shape[0]):
    a, b =generate_image2(kor[i], kor_label[i])
    new_k = np.concatenate((a, new_k))
    new_k_lable = np.concatenate((b, new_k_lable))


In [0]:
#데이터가 40배가 된 것을 확인
print("공유폴더 한국인 이미지 원본:",kor.shape)
print("40배로 증가된 이미지: ",new_k.shape)

In [0]:
#증가된 유러피안과 한국인 데이터를 모두 합침
x = np.concatenate((new_k, x_train))
y = np.concatenate((new_k_lable, y_train))

In [0]:
#검증셋과 훈련셋으로 나눔
x, x_val, y, y_val = train_test_split(x, y, test_size=0.1,random_state = 64)

In [0]:
#증가된 한국인데이터와 유러피안 데이터로 학습 및 평가
model3 = make_model()
model_compile(model3,0.00008)    #lr = 0.00008
history = model3.fit(x, y,
        batch_size=256,
        epochs=100,
        validation_data=(x_val, y_val),
        callbacks = [early_stopping],   #조기종료함수
        verbose=1)
show_history(history)
model_eval(model3)

In [0]:
model2.summary()

In [0]:
loss, accuracy = model2.evaluate(kx_test, ky_test, verbose = 1)
print("loss: ",loss)
print("accuracy: ",accuracy*100,'%')

In [0]:
loss, accuracy = model2.evaluate(x_test, y_test, verbose = 1)
print("loss: ",loss)
print("accuracy: ",accuracy*100,'%')

#새로운 모델구조 만들어보기

In [0]:
input_shape = (48, 48, 1)
   
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=input_shape, activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(7, activation='softmax'))

In [0]:
model_compile(model,0.0001)    
history = model.fit(x, y,
        batch_size=256,
        epochs=50,
        validation_data=(x_val, y_val),
        callbacks = [early_stopping],   #조기종료함수
        verbose=1)
show_history(history)
model_eval(model)

In [0]:
loss, accuracy = model.evaluate(x_test, y_test, verbose = 1)
print("loss: ",loss)
print("accuracy: ",accuracy*100,'%')

In [0]:
loss, accuracy = model.evaluate(kx_test, ky_test, verbose = 1)
print("loss: ",loss)
print("accuracy: ",accuracy*100,'%')